In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nahuatl-spanish/test.nah-es.src
/kaggle/input/nahuatl-spanish/test.es-nah.src
/kaggle/input/nahuatl-spanish/train.es
/kaggle/input/nahuatl-spanish/dev.nah
/kaggle/input/nahuatl-spanish/dev.es
/kaggle/input/nahuatl-spanish/train.nah


In [2]:
%%capture
%pip install sentencepiece transformers==4.33 datasets sacremoses sacrebleu -q

Import libraries

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, NllbTokenizer
from transformers.optimization import Adafactor
from transformers import get_constant_schedule_with_warmup
from sklearn.model_selection import train_test_split
import re
import random
from tqdm.auto import tqdm
import os

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HUGGINGFACE_TOKEN = user_secrets.get_secret("HUGGINGFACE_TOKEN")

!huggingface-cli login --token $HUGGINGFACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `basic task` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `basic task`


## Dataset

In [5]:
# Initialize lists to store the lines from each file
ashaninka_lines = []
spanish_lines = []
source = []

# Read train.cni (Ashaninka) file
with open('/kaggle/input/nahuatl-spanish/train.nah', 'r', encoding='utf-8') as file:
    ashaninka_lines = [line.strip() for line in file]

# # Read train.cni (Ashaninka) file
# with open('/kaggle/input/awajun-spanish/dev.source', 'r', encoding='utf-8') as file:
#     source = [line.strip() for line in file]

# Read train.es (Spanish) file
with open('/kaggle/input/nahuatl-spanish/train.es', 'r', encoding='utf-8') as file:
    spanish_lines = [line.strip() for line in file]

# Create a DataFrame
train_df  = pd.DataFrame({
    # 'Source': source,
    'Nahuatl': ashaninka_lines,
    'Spanish': spanish_lines
})

In [6]:
# Initialize lists to store the lines from each file
ashaninka_lines = []
spanish_lines = []
source = []

# Read train.cni (Ashaninka) file
with open('/kaggle/input/nahuatl-spanish/dev.nah', 'r', encoding='utf-8') as file:
    ashaninka_lines = [line.strip() for line in file]

# # Read train.cni (Ashaninka) file
# with open('/kaggle/input/awajun-spanish/dev.source', 'r', encoding='utf-8') as file:
#     source = [line.strip() for line in file]

# Read train.es (Spanish) file
with open('/kaggle/input/nahuatl-spanish/dev.es', 'r', encoding='utf-8') as file:
    spanish_lines = [line.strip() for line in file]

# Create a DataFrame
dev_df  = pd.DataFrame({
    # 'Source': source,
    'Nahuatl': ashaninka_lines,
    'Spanish': spanish_lines
})

In [7]:
# Create a test set from the dev data
dev_df, test_df = train_test_split(dev_df, test_size=0.5, random_state=42)

In [8]:
print(f"Training samples: {len(train_df)}")
print(f"Development samples: {len(dev_df)}")
print(f"Test samples: {len(test_df)}")

Training samples: 16145
Development samples: 336
Test samples: 336


In [9]:
# Step 4: Initialize the NLLB tokenizer and model
model_name = "facebook/nllb-200-distilled-600M"
tokenizer = NllbTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=map

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

## Examine tokenization

In [10]:
def word_tokenize(text):
    return re.findall('(\w+|[^\w\s])', text)

In [11]:
# Sample from the training data
sample = train_df.sample(min(10000, len(train_df)), random_state=1)

In [12]:
# Apply tokenization
sample['nah_toks'] = sample.Nahuatl.apply(tokenizer.tokenize)
sample['spa_toks'] = sample.Spanish.apply(tokenizer.tokenize)
sample['nah_words'] = sample.Nahuatl.apply(word_tokenize)
sample['spa_words'] = sample.Spanish.apply(word_tokenize)

In [13]:
# Compute statistics
stats = sample[['nah_toks', 'spa_toks', 'nah_words', 'spa_words']].applymap(len).describe()

<ipython-input-13-f5f92b4ff816>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  stats = sample[['nah_toks', 'spa_toks', 'nah_words', 'spa_words']].applymap(len).describe()


In [14]:
print("Average tokens per word for Wayuu:", stats.nah_toks['mean'] / stats.nah_words['mean'])
print("Average tokens per word for Spanish:", stats.spa_toks['mean'] / stats.spa_words['mean'])

Average tokens per word for Wayuu: 2.3097126684191727
Average tokens per word for Spanish: 1.3518645947805725


In [15]:
# Check for unknown tokens
texts_with_unk_nah = [
    text for text in tqdm(train_df.Nahuatl)
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]

texts_with_unk_spa = [
    text for text in tqdm(train_df.Spanish)
    if tokenizer.unk_token_id in tokenizer(text).input_ids
]

  0%|          | 0/16145 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1041 > 1024). Running this sequence through the model will result in indexing errors


  0%|          | 0/16145 [00:00<?, ?it/s]

In [16]:
print(f"Number of Wayuu texts with unknown tokens: {len(texts_with_unk_nah)}")
print(f"Number of Spanish texts with unknown tokens: {len(texts_with_unk_spa)}")

Number of Wayuu texts with unknown tokens: 406
Number of Spanish texts with unknown tokens: 724


## Add Language Token if needed

In [17]:
def fix_tokenizer(tokenizer, new_lang='nah_Latn'):
    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset
    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

In [18]:
if 'nah_Latn' not in tokenizer.lang_code_to_id:
    fix_tokenizer(tokenizer, 'nah_Latn')
    model.resize_token_embeddings(len(tokenizer))
    
    # Initialize the embedding for the new token with a similar language
    added_token_id = tokenizer.convert_tokens_to_ids('nah_Latn')
    similar_lang_id = tokenizer.convert_tokens_to_ids('quy_Latn')  # Quechua is similar to Aymara
    embeds = model.model.shared.weight.data
    embeds[added_token_id] = embeds[similar_lang_id]

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 256205. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


## Preprocess function for text(Normalizer)

In [19]:
def preproc(text):
    from sacremoses import MosesPunctNormalizer
    mpn = MosesPunctNormalizer(lang="en")
    clean = mpn.normalize(text)
    return clean

## Training function

In [20]:
def get_batch_pairs(batch_size, data=train_df):
    LANGS = [('Nahuatl', 'nah_Latn'), ('Spanish', 'spa_Latn')]
    (l1, long1), (l2, long2) = random.sample(LANGS, 2)
    xx, yy = [], []
    for _ in range(batch_size):
        item = data.iloc[random.randint(0, len(data)-1)]
        xx.append(preproc(item[l1]))
        yy.append(preproc(item[l2]))
    return xx, yy, long1, long2

In [21]:
def train_model(model, tokenizer, training_steps=20000, batch_size=16, max_length=128, lr=1e-4):
    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Define optimizer and scheduler
    optimizer = Adafactor(
        [p for p in model.parameters() if p.requires_grad],
        scale_parameter=False,
        relative_step=False,
        lr=lr,
        clip_threshold=1.0,
        weight_decay=1e-3,
    )
    scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=1000)
    
    # Training loop
    losses = []
    MODEL_SAVE_PATH = 'nllb-nah-spa-v1'
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    
    model.train()
    x, y, loss = None, None, None
    
    def cleanup():
        import gc
        gc.collect()
        torch.cuda.empty_cache()
    
    for i in tqdm(range(training_steps)):
        xx, yy, lang1, lang2 = get_batch_pairs(batch_size)
        
        try:
            tokenizer.src_lang = lang1
            x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
            tokenizer.src_lang = lang2
            y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)
            y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100
            
            loss = model(**x, labels=y.input_ids).loss
            loss.backward()
            losses.append(loss.item())
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
            
        except RuntimeError as e:
            optimizer.zero_grad(set_to_none=True)
            x, y, loss = None, None, None
            cleanup()
            print('error', max(len(s) for s in xx + yy), e)
            continue
        
        if i % 1000 == 0:
            avg_loss = torch.mean(torch.tensor(losses[-1000:] if len(losses) > 1000 else losses))
            print(f"Step {i}, Average Loss: {avg_loss}")
            if i % 5000 == 0 and i > 0:
                model.save_pretrained(f"{MODEL_SAVE_PATH}/checkpoint-{i}")
                tokenizer.save_pretrained(f"{MODEL_SAVE_PATH}/checkpoint-{i}")
    
    # Save the final model
    model.save_pretrained(MODEL_SAVE_PATH)
    tokenizer.save_pretrained(MODEL_SAVE_PATH)
    
    return model, tokenizer


## Evaluation function

In [22]:
def evaluate_model(model, tokenizer, test_data):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    # Function to translate text
    def translate(text, src_lang, tgt_lang, num_beams=4):
        tokenizer.src_lang = src_lang
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=1024)
        with torch.no_grad():
            outputs = model.generate(
                **inputs.to(device), 
                forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang), 
                num_beams=num_beams
            )
        return tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # Translate Aymara to Spanish
    nah_to_spa = []
    for text in tqdm(test_data.Nahuatl):
        nah_to_spa.append(translate(text, 'nah_Latn', 'spa_Latn')[0])
    
    # Translate Spanish to Aymara
    spa_to_nah = []
    for text in tqdm(test_data.Spanish):
        spa_to_nah.append(translate(text, 'spa_Latn', 'nah_Latn')[0])
    
    # Calculate BLEU scores
    import sacrebleu
    
    bleu_nah_spa = sacrebleu.corpus_bleu(nah_to_spa, [test_data.Spanish.tolist()])
    bleu_spa_nah = sacrebleu.corpus_bleu(spa_to_nah, [test_data.Nahuatl.tolist()])
    
    chrf_nah_spa = sacrebleu.corpus_chrf(nah_to_spa, [test_data.Spanish.tolist()])
    chrf_spa_nah = sacrebleu.corpus_chrf(spa_to_nah, [test_data.Nahuatl.tolist()])
    
    # Calculate exact match accuracy
    nah_spa_exact_matches = sum(1 for pred, ref in zip(nah_to_spa, test_data.Spanish) if pred.strip() == ref.strip())
    spa_nah_exact_matches = sum(1 for pred, ref in zip(spa_to_nah, test_data.Nahuatl) if pred.strip() == ref.strip())
    
    nah_spa_accuracy = (nah_spa_exact_matches / len(test_data)) * 100
    spa_nah_accuracy = (spa_nah_exact_matches / len(test_data)) * 100
    
    # Calculate word-level accuracy (percentage of words that match)
    def calculate_word_accuracy(predictions, references):
        total_words = 0
        correct_words = 0
        
        for pred, ref in zip(predictions, references):
            pred_words = pred.strip().split()
            ref_words = ref.strip().split()
            
            # Count words in reference
            total_words += len(ref_words)
            
            # Count matching words (position-sensitive)
            for i, ref_word in enumerate(ref_words):
                if i < len(pred_words) and pred_words[i] == ref_word:
                    correct_words += 1
        
        return (correct_words / total_words) * 100 if total_words > 0 else 0
    
    nah_spa_word_accuracy = calculate_word_accuracy(nah_to_spa, test_data.Spanish)
    spa_nah_word_accuracy = calculate_word_accuracy(spa_to_nah, test_data.Nahuatl)
    
    print(f"BLEU Nahuatl → Spanish: {bleu_nah_spa.score:.2f}")
    print(f"BLEU Spanish → Nahuatl: {bleu_spa_nah.score:.2f}")
    print(f"chrF++ Nahuatl → Spanish: {chrf_nah_spa.score:.2f}")
    print(f"chrF++ Spanish → Nahuatl: {chrf_spa_nah.score:.2f}")
    print(f"Exact Match Accuracy Nahuatl → Spanish: {nah_spa_accuracy:.2f}%")
    print(f"Exact Match Accuracy Spanish → Nahuatl: {spa_nah_accuracy:.2f}%")
    print(f"Word-level Accuracy Nahuatl → Spanish: {nah_spa_word_accuracy:.2f}%")
    print(f"Word-level Accuracy Spanish → Nahuatl: {spa_nah_word_accuracy:.2f}%")
    
    return {
        'bleu_nah_spa': bleu_nah_spa.score,
        'bleu_spa_nah': bleu_spa_nah.score,
        'chrf_nah_spa': chrf_nah_spa.score,
        'chrf_spa_nah': chrf_spa_nah.score,
        'nah_spa_accuracy': nah_spa_accuracy,
        'spa_nah_accuracy': spa_nah_accuracy,
        'nah_spa_word_accuracy': nah_spa_word_accuracy,
        'spa_nah_word_accuracy': spa_nah_word_accuracy,
        'nah_to_spa_samples': list(zip(test_data.Nahuatl.head(5), nah_to_spa[:5])),
        'spa_to_nah_samples': list(zip(test_data.Spanish.head(5), spa_to_nah[:5]))
    }

## Main Execution

In [23]:
# Train the model
print("Starting model training...")
model, tokenizer = train_model(model, tokenizer, training_steps=20000) #Min 10000 steps need to be a good translator

Starting model training...


  0%|          | 0/20000 [00:00<?, ?it/s]

Step 0, Average Loss: 3.9045090675354004
Step 1000, Average Loss: 3.84911847114563
Step 2000, Average Loss: 2.74861741065979
Step 3000, Average Loss: 2.3282134532928467
Step 4000, Average Loss: 2.0634427070617676
Step 5000, Average Loss: 1.8682842254638672
Step 6000, Average Loss: 1.7349060773849487
Step 7000, Average Loss: 1.5998868942260742
Step 8000, Average Loss: 1.4857168197631836
Step 9000, Average Loss: 1.391996145248413
Step 10000, Average Loss: 1.2904860973358154
Step 11000, Average Loss: 1.224064588546753
Step 12000, Average Loss: 1.129186987876892
Step 13000, Average Loss: 1.0682711601257324
Step 14000, Average Loss: 0.996343731880188
Step 15000, Average Loss: 0.9390100836753845
Step 16000, Average Loss: 0.8929599523544312
Step 17000, Average Loss: 0.8226222991943359
Step 18000, Average Loss: 0.7825664281845093
Step 19000, Average Loss: 0.7237638235092163


## Hugging Face Model Push

In [24]:
model_name = "mahsharyahan/nllb-600B-nah-spa-v1"
tokenizer.push_to_hub(model_name)
model.push_to_hub(model_name)

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mahsharyahan/nllb-600B-nah-spa-v1/commit/272275a48cab134b1f1f3fb96582b641e3713528', commit_message='Upload M2M100ForConditionalGeneration', commit_description='', oid='272275a48cab134b1f1f3fb96582b641e3713528', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mahsharyahan/nllb-600B-nah-spa-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='mahsharyahan/nllb-600B-nah-spa-v1'), pr_revision=None, pr_num=None)

In [25]:
# Evaluate the model
print("Evaluating the model...")
eval_results = evaluate_model(model, tokenizer, test_df)

Evaluating the model...


  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

BLEU Nahuatl → Spanish: 7.22
BLEU Spanish → Nahuatl: 1.31
chrF++ Nahuatl → Spanish: 26.89
chrF++ Spanish → Nahuatl: 29.92
Exact Match Accuracy Nahuatl → Spanish: 0.00%
Exact Match Accuracy Spanish → Nahuatl: 0.00%
Word-level Accuracy Nahuatl → Spanish: 10.01%
Word-level Accuracy Spanish → Nahuatl: 3.65%


## Sample Testing

In [26]:
# Print some translation examples
print("\Awajun to Spanish translation examples:")
for src, tgt in eval_results['nah_to_spa_samples']:
    print(f"Source: {src}")
    print(f"Translation: {tgt}")
    print()

print("\nSpanish to Awajun translation examples:")
for src, tgt in eval_results['spa_to_nah_samples']:
    print(f"Source: {src}")
    print(f"Translation: {tgt}")
    print()

\Awajun to Spanish translation examples:
Source: tehtepitzin itla quicuepqueh nochi tlen nicchihuaznequia
Translation: a poco algo le devolvieron todo lo que yo quería hacer

Source: Naman nicpiya miyac tomitzin, nimotlacohuiti.
Translation: Mamá tiene mucho dinero, lo compré.

Source: huan melan amo quimatic
Translation: y si no saben nada

Source: Na zan nictequihuia tilahuac noyoyon queman nitequiti nochan.
Translation: Yo sólo trabajo al medio día mientras que trabajo en mi casa.

Source: Ramona yetoc concord
Translation: " Ramona, quinto, de acuerdo


Spanish to Awajun translation examples:
Source: Pequeñas cosas como esta marcaron una gran diferencia en lo que estaba tratando de hacer.
Translation: Zazotzitzin inin no iuhqui cenca tlapanahuia itechpa tlen quichihuaznequia.

Source: Tengo un montón de dinero en este momento, así que quiero ir de compras.
Translation: Nicpia miac tomin axcan, iqui in nicnequi niaz tlalnamacoyan.

Source: Y en realidad no lo entendió.
Translation: U

## Submit Function

In [27]:
def test_translation_without_targets(model, tokenizer, test_data, src_lang, tgt_lang, num_samples=5):
    import torch
    from tqdm.auto import tqdm
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    # Determine column name based on language direction
    if src_lang == 'nah_Latn':
        src_col = 'Nahuatl'
    else:
        src_col = 'Spanish'
    
    # Function to translate text
    def translate(text, num_beams=4, max_length=128):
        tokenizer.src_lang = src_lang
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_length)
        with torch.no_grad():
            outputs = model.generate(
                **inputs.to(device), 
                forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang), 
                num_beams=num_beams,
                max_length=max_length
            )
        return tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    # Translate test samples
    translations = []
    
    # Select random samples for detailed review
    test_samples = test_data.sample(min(num_samples, len(test_data)))
    
    # Process all test data
    print(f"\n===== Translation Results: {src_lang} to {tgt_lang} =====")
    print("Sample Translations:")
    
    for i, (_, row) in enumerate(tqdm(test_data.iterrows(), total=len(test_data), desc=f"Translating {src_lang} to {tgt_lang}")):
        source_text = row[src_col]
        translated_text = translate(source_text)[0]
        translations.append(translated_text)
        
        # Display detailed examples for the selected samples
        if _ in test_samples.index:
            print(f"\nExample {i+1}:")
            print(f"Source ({src_col}): {source_text}")
            print(f"Translation: {translated_text}")
    
    return translations

In [28]:
# Initialize lists to store the lines from each file
ashaninka_lines = []

# Read train.cni (Ashaninka) file
with open('/kaggle/input/nahuatl-spanish/test.es-nah.src', 'r', encoding='utf-8') as file:
    ashaninka_lines = [line.strip() for line in file]

# Create a DataFrame
df_test_nah_es = pd.DataFrame({
    # 'Source': source,
    'Nahuatl': ashaninka_lines,
})

In [29]:
nah_to_es_results = test_translation_without_targets(model, tokenizer, df_test_nah_es, 'nah_Latn', 'spa_Latn')


===== Translation Results: nah_Latn to spa_Latn =====
Sample Translations:


Translating nah_Latn to spa_Latn:   0%|          | 0/1003 [00:00<?, ?it/s]


Example 462:
Source (Nahuatl): Dijo que bajaron al sur.
Translation: Dijo que bajaron al sur.

Example 494:
Source (Nahuatl): Soy ignorante en asuntos de amor.
Translation: Soy ignorante en asuntos de amor.

Example 577:
Source (Nahuatl): no somos verdaderos naturalistas ni nada pero eh
Translation: también somos verdaderos naturalistas nada más

Example 671:
Source (Nahuatl): Voy a tener que hacerlo, ya que están llegando a la edad.
Translation: Voy a tener que hacerlo, ya que están llegando a la edad.

Example 731:
Source (Nahuatl): un níquel menos, o lo que es lo mismo, treinta centavos menos por seis botellas. Todos cruzan la frontera para comprar las bebidas más baratas
Translation: un niñito menos, o lo mismo 30 centavos menos por seis botellas. Todos cruzan la frontera para comprar las bebidas más baratas.


In [30]:
# Create a DataFrame
df_test_result_nah_to_es = pd.DataFrame({
    # 'Source': source,
    'Spanish': nah_to_es_results,
})

In [31]:
df_test_result_nah_to_es.to_csv('test_result_nah_to_es.tsv', sep='\t', index=False, header=False)

In [32]:
from IPython.display import FileLink

FileLink("test_result_nah_to_es.tsv")

/kaggle/working/test_result_nah_to_es.tsv

In [33]:
# Initialize lists to store the lines from each file
ashaninka_lines = []

# Read train.cni (Ashaninka) file
with open('/kaggle/input/nahuatl-spanish/test.nah-es.src', 'r', encoding='utf-8') as file:
    ashaninka_lines = [line.strip() for line in file]

# Create a DataFrame
df_test_es_nah = pd.DataFrame({
    # 'Source': source,
    'Spanish': ashaninka_lines,
})

In [34]:
es_to_nah_results = test_translation_without_targets(model, tokenizer, df_test_es_nah, 'spa_Latn','nah_Latn')


===== Translation Results: spa_Latn to nah_Latn =====
Sample Translations:


Translating spa_Latn to nah_Latn:   0%|          | 0/1003 [00:00<?, ?it/s]


Example 370:
Source (Spanish): huahca aman, nicac eltoc, aman quinequi
Translation: uajka kiitstoke nikak uajkaya kiitstoke kinekij

Example 383:
Source (Spanish): amo nihmati quesqui quipiya.
Translation: axkimati kenijki kipiya.

Example 593:
Source (Spanish): Ino, nazantzi nipiyaya 922 tlen tlacatl de soporte vital, nompa tlacatl eltoya tepalehuihquetl tlen ficiologico.
Translation: Inic ontlamantli onicac ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin ontlamantin

Example 660:
Source (Spanish): Hum, ya cihuatl quiihtohqui...quiihtohqui…quiihtohqui, noyoltzi, quiihtohqui, axticuamachilia nemiliztli quema na nicuamachilia.
Translation: hummer ye cihuatl quiihtohua quiihtohua 

In [35]:
# Create a DataFrame
df_test_result_es_to_nah = pd.DataFrame({
    # 'Source': source,
    'Nahuatl': es_to_nah_results,
})

In [36]:
df_test_result_es_to_nah.to_csv('test_result_es_to_nah.tsv', sep='\t', index=False, header=False)

In [37]:
from IPython.display import FileLink

FileLink("test_result_es_to_nah.tsv")

/kaggle/working/test_result_es_to_nah.tsv